In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [ ]:
chromedriver = 'D:/20220103_lab/14.python/data/chrome/chromedriver'
driver = webdriver.Chrome(chromedriver) 

# 포스팅 작성 당시 크롬 버젼 : 92

In [ ]:
df = pd.DataFrame

In [ ]:
# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''

In [ ]:
# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

In [ ]:
# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [ ]:
# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

In [ ]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
Python


In [ ]:
from selenium import webdriver
import time

# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

In [ ]:
# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

In [ ]:
# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [53]:
# 여러 페이지(999)에서 반복하기
for i in range(999):
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")
    

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

            # 가게 주소 데이터 수집 // span.section-result-location

            addr = s.find_element_by_css_selector("span.section-result-location").text

            print(title, "/", score, "/", addr)
            
    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2762349343.py:6: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  stores = driver.find_elements_by_css_selector("div.section-result-content")
C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2762349343.py:28: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")


데이터 수집 완료.


In [54]:
stores

[]

In [ ]:
# 크롬창 닫기
driver.close()
# 출처: https://datalabbit.tistory.com/45 [간토끼 DataMining Lab:티스토리]

In [18]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [19]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [20]:
driver.get("https://www.google.com/maps/")

In [21]:
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2243747384.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  searchbox = driver.find_element_by_css_selector("input#searchboxinput")


In [22]:
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1190549724.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")


In [126]:
for i in range(999):
    time.sleep(3)
    stores = driver.find_elements_by_css_selector("QA0Szd")
    for s in stores:
        print(s)
        title = s.find_element_by_css_selector("h3.section-result-title").text
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"
            
            addr = s.find_element_by_css_selector("span.section-result-location").text
            
            print(title, "/", score, "/", addr)
            
    try:
        nextpage = driver.find_element_by_css_selebctor("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

데이터 수집 완료.


C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1891611790.py:3: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  stores = driver.find_elements_by_css_selector("QA0Szd")


In [65]:
s

NameError: name 's' is not defined

In [63]:
title

NameError: name 'title' is not defined

In [ ]:
driver.close()

In [33]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [34]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [35]:
driver.get("https://www.google.com/maps/")

In [36]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(1)

In [37]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [66]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]')

In [39]:
target.text

'평점\n영업시간\n모든 필터\n아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n지금 영업 중 · 032-553-4652\n웹사이트\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582\n웹사이트\n경로\n이지동물병원\n4.5(8)\n동물 병원 · 병방동 422\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-549-0077\n경로\n신통방통동물병원\n4.9(10)\n동물 병원 · 작전동 863\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-552-3369\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n작전24동물병원\n4.4(7)\n동물병원 · 작전동 925-1\n경로\n재영동물병원\n3.8(4)\n동물병원 · 계산1동\n지금 영업 중 · 032-542-4647\n경로\n검색결과 1 - 20 표시 중\n지도 이동 시 결과 업데이트\n로그인\n레이어\n지도 데이터 ©2022 TMap Mobility대한민국약관개인정보처리방침\n의견 보내기\n500m'

In [170]:
driver.execute_script("window.scrollTo(0, 10000)") 

In [171]:
driver.execute_script("window.scrollTo(0, 1000)") 

In [43]:
last_height = driver.execute_script("return document.body.scrollHeight")

In [44]:
last_height

887

In [45]:
SCROLL_PAUSE_SEC = 1
while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [50]:
# ActionChains 를 사용하기 위해서.
from selenium.webdriver import ActionChains

# id가 something 인 element 를 찾음
some_tag = driver.find_element(By.ID, 'pane')

# somthing element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(some_tag).perform()


In [8]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

JavascriptException: Message: javascript error: Cannot read properties of null (reading 'scrollTo')
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x009FF154+848212]
	Ordinal0 [0x009FF012+847890]
	Ordinal0 [0x009FF98A+850314]
	Ordinal0 [0x00A550C9+1200329]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [ ]:
for divtag in target.find_elements(By.XPATH, './div'):
    if divtag.text != "":
        print("--------------------------")
        atag = divtag.find_element(By.XPATH, './div/a[1]')
        subdiv = divtag.find_element(By.XPATH, "./div/div[2]")
        name = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
        star = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
        addr = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[1]")

        print(name.text)
        print(addr.text)
        print(atag.get_attribute('href').split('!')[5][2:])
        print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
        print(star.text)
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

In [144]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_20184/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [4]:
driver.get("https://map.kakao.com/")

In [5]:
search_input_tag = driver.find_element(By.ID, 'search.keyword.query')
search_input_tag.send_keys('동물병원')
time.sleep(1)

In [6]:
search_button_tag = driver.find_element(By.ID, 'search.keyword.submit')
search_button_tag.click()
time.sleep(5)

In [8]:
place_more = driver.find_element(By.ID, 'info.search.place.more')
place_more.click()
time.sleep(5)

In [40]:
for i in range(999):
    nextpage = driver.find_element(By.ID, "info.search.page.no{}".format((i+1)%5))
    nextpage.click()
    print(i)
    if i == 4 :
        nextpage = driver.find_element(By.ID, "info.search.page.next").click()
        time.sleep(1)
        i = 1

0
1
2
3
4


In [223]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [202]:
search_input_tag = driver.find_element(By.CLASS_NAME, 'mL3xi')
search_input_tag.send_keys('동물병원')
time.sleep(3)


In [253]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")
driver.get("https://www.google.com/maps/")
time.sleep(3)

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/451005478.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [254]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(3)

In [256]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

ElementClickInterceptedException: Message: element click intercepted: Element <button aria-label="검색" id="searchbox-searchbutton" jstcache="13" jsaction="omnibox.search;mouseover:omnibox.showTooltip;mouseout:omnibox.hideTooltip;focus:omnibox.showTooltip;blur:omnibox.hideTooltip" class="mL3xi" jsan="7.mL3xi,0.aria-label,0.id,22.jsaction"></button> is not clickable at point (319, 32). Other element would receive the click: <canvas width="1920" height="924" id="" class="MyME0d widget-scene-canvas" style="width: 1920px; height: 924px;"></canvas>
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A2FA1B+1047067]
	Ordinal0 [0x00A2D9F8+1038840]
	Ordinal0 [0x00A2B6DB+1029851]
	Ordinal0 [0x00A2A439+1025081]
	Ordinal0 [0x00A20083+983171]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096

In [229]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")
driver.get("https://www.google.com/maps/")
time.sleep(3)
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(3)
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/696269896.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


KeyboardInterrupt: 

In [259]:
driver.execute_script("document.body.style.zoom='100%'")

#elem.send_keys(Keys.ENTER)

In [257]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[2]')

In [258]:
target.text

'아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-9975\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546-0079\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n032-553-4652\n웹사이트\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582\n웹사이트\n경로\n부평종합동물병원\n4.5(101)\n동물병원 · 부평동 부평대로 154\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-511-6836\n경로\n보비동물병원\n4.6(48)\n포근한동물병원\n4.9(31)\n동물병원 · 중동 1168\n24시간 영업 · 032-231-0075\n경로\n펫사랑동물병원\n4.4(48)\n동물병원 · 산곡동 194-6\n032-521-0003\n웹사이트\n경로\n24시나음동물병원\n4.6(45)\n동물 병원 · 중동\n032-323-7581\n경로\n부천 TLC 동물의료센터\n4.9(35)\n동물병원 · 중동 길주로 281 TLC 동물의료센터 1층\n24시간 영업 · 032-323-2475\n웹사이트\n경로\n동행동물병원\n4.7(52)\n동물병원 · 양도로 19번길 15\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 031-986-9816\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n신통방통동물병원\n4.9(10)\n동물 

In [162]:
panee = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[8]')

class = "m6QErb DxyBCb kA9KIf dS8AEf ecceSd"

In [193]:
asdasdf = driver.find_element_by_css_selector("div.rHNip.cRLbXd")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1298259985.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  asdasdf = driver.find_element_by_css_selector("div.rHNip.cRLbXd")


In [191]:
asdasdf.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A2A2D3+1024723]
	Ordinal0 [0x00A20083+983171]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [194]:
asdasdf.send_keys(Keys.END)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A249C3+1001923]
	Ordinal0 [0x00A240A5+999589]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [203]:
akjsvebjkseagvf2 = driver.find_element_by_class_name("rHNip cRLbXd")


C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3682334982.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  akjsvebjkseagvf2 = driver.find_element_by_class_name("rHNip cRLbXd")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rHNip cRLbXd"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A56372+1205106]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [208]:
akjsvebjkseagvf2 = driver.find_element_by_css_selector("div.drGLxe")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3934446620.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  akjsvebjkseagvf2 = driver.find_element_by_css_selector("div.drGLxe")


In [212]:
akjsvebjkseagvf2.send_keys(Keys.ENTER)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A249C3+1001923]
	Ordinal0 [0x00A240A5+999589]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [205]:
akjsvebjkseagvf = driver.find_element(By.CLASS_NAME, "tXNTee LCTIRd L6Bbsd VbYL9")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".tXNTee LCTIRd L6Bbsd VbYL9"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A56372+1205106]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [195]:
jksdfkhsbdf = driver.find_element_by_css_selector("div.drGLxe")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2244094299.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  jksdfkhsbdf = driver.find_element_by_css_selector("div.drGLxe")


In [196]:
jksdfkhsbdf.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A2A2D3+1024723]
	Ordinal0 [0x00A20083+983171]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [153]:
target.text

'평점\n영업시간\n모든 필터\n아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n032-553-4652\n웹사이트\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582\n웹사이트\n경로\n이지동물병원\n4.5(8)\n동물 병원 · 병방동 422\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-549-0077\n경로\n신통방통동물병원\n4.9(10)\n동물 병원 · 작전동 863\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-552-3369\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n작전24동물병원\n4.4(7)\n동물병원 · 작전동 925-1\n경로\n재영동물병원\n3.8(4)\n동물병원 · 계산1동\n032-542-4647\n경로\n검색결과 1 - 20 표시 중\n지도 이동 시 결과 업데이트'

In [187]:
target.send_keys(Keys.END)

In [155]:
driver.execute_script("window.scrollTo(0, 10000)") 

In [159]:
driver.execute_script('arguments[0].scrollIntoView(true);', target)

In [169]:
actions = ActionChains(driver) \
    .move_to_element(panee) \
    .click() \
    .send_keys(Keys.END)

actions.perform() 

In [ ]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

divtag -> class -> "m6QErb DxyBCb kA9KIf dS8AEf ecceSd QjC7t"

subdiv -> jstcache="184" 들 모아놓음

name -> class="NrDZNb"

star -> jstcache="83"

In [ ]:
lname = []
lstar = []
laddr = []

In [260]:
for divtag in target.find_elements(By.XPATH, './div[1]'):
    if divtag.text != "":
        for i in lista:
            print("--------------------------")
            atag = divtag.find_element(By.XPATH, './div[{}]/div[1]/a[1]'.format(i))
            subdiv = divtag.find_element(By.XPATH, "./div[{}]".format(i))
            name = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
            star = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
            addr = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[1]/span[2]")

            print(name.text)
            print(star.text)
            print(addr.text.replace("· ",""))
            
#            print(atag.get_attribute('href').split('!')[5][2:])
#            print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
            
            lname.append(name.text)
            lstar.append(star.text)
            laddr.append(addr.text.replace("· ",""))
            
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

--------------------------
지중해동물병원
4.8(34)
중동 1058-2번지 107호 중동프라자 부천시 경기도 KR
--------------------------
웰케어동물메디컬센터
4.1(34)
풍무동 592-10
--------------------------
청라호수동물병원
4.2(51)
푸른로7번길 7-2
--------------------------
청라루비동물병원
4.3(34)
청라동 루비로134번길 6-4
--------------------------
하얀기린동물병원
5.0(32)
연희동 796-7번지 청라골드클래스커낼웨이 상가동 103호 서구 인천광역시 KR
--------------------------
가정동물병원
4.0(43)
가정1동 517-32
--------------------------
작전24동물병원
4.4(7)
작전동 925-1
--------------------------
동수동물병원
4.7(18)
부평1동 182-10
--------------------------
중동동물병원
4.6(21)
중동 749-17
--------------------------
비엔동물전문의료센터
4.5(95)
경인로 475
--------------------------
위더스동물병원
4.4(13)
상3동 534-9
--------------------------
삼성동물병원
4.1(26)
석남2동 579-3
--------------------------
주주종합동물병원
4.8(9)
부개1동 301-38
--------------------------
세명동물병원
4.7(10)
화곡1동 353-58
--------------------------
파스텔동물병원
4.7(20)
화곡1동 1070-1
--------------------------
부평모모동물병원
4.9(10)
경원대로 1278 1층
--------------------------
온유동물병원
4.0(19)
중동 1087
-----------------

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[43]/div[1]/a[1]"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A1FAE1+981729]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x7

In [84]:
target.find_elements(By.XPATH, './div')

[<selenium.webdriver.remote.webelement.WebElement (session="8144a835305002e05f47710ddfab6e45", element="a2eeba5a-9774-4e01-bdeb-cf1cb487556f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8144a835305002e05f47710ddfab6e45", element="891b35ab-812c-4838-83fa-f1b8ba297136")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8144a835305002e05f47710ddfab6e45", element="0b12d376-226d-4d47-aff4-c1a125669bd0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8144a835305002e05f47710ddfab6e45", element="d7f340fe-3cbe-446f-b911-e1f7da9385fc")>]

In [99]:
lista = []
for x in list(range(50)):
    if x % 2 :
        lista.append(x)

In [102]:
lista

[3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49]

In [261]:
lname

['아마존동물병원',
 '계양동물메디컬센터',
 '한솔동물병원',
 '해림동물병원',
 '이지동물병원',
 '신통방통동물병원',
 '다오동물병원',
 '재영동물병원',
 '신영재동물병원',
 '작전24동물병원',
 '아프리카 동물의료센터 [구 아프리카동물병원]',
 '부평종합동물병원',
 '보비동물병원',
 '펫가든동물병원',
 '부평종합동물의료센터',
 '세림동물병원',
 '닥터팬더동물병원',
 '포근한동물병원',
 '부천준동물병원',
 '펫사랑동물병원',
 '지중해동물병원',
 '웰케어동물메디컬센터',
 '청라호수동물병원',
 '청라루비동물병원',
 '하얀기린동물병원',
 '가정동물병원',
 '작전24동물병원',
 '동수동물병원',
 '중동동물병원',
 '비엔동물전문의료센터',
 '위더스동물병원',
 '삼성동물병원',
 '주주종합동물병원',
 '세명동물병원',
 '파스텔동물병원',
 '부평모모동물병원',
 '온유동물병원',
 '하노바동물병원',
 '청라종합동물병원',
 '도그앤캣동물병원']

In [262]:
lstar

['4.6(25)',
 '4.3(50)',
 '4.3(13)',
 '4.8(100)',
 '4.5(8)',
 '4.9(10)',
 '4.7(6)',
 '3.8(4)',
 '4.0(2)',
 '4.4(7)',
 '4.4(89)',
 '4.5(101)',
 '4.6(48)',
 '5.0(1)',
 '4.4(48)',
 '5.0(23)',
 '리뷰 없음',
 '4.9(31)',
 '4.8(27)',
 '4.4(48)',
 '4.8(34)',
 '4.1(34)',
 '4.2(51)',
 '4.3(34)',
 '5.0(32)',
 '4.0(43)',
 '4.4(7)',
 '4.7(18)',
 '4.6(21)',
 '4.5(95)',
 '4.4(13)',
 '4.1(26)',
 '4.8(9)',
 '4.7(10)',
 '4.7(20)',
 '4.9(10)',
 '4.0(19)',
 '4.6(11)',
 '5.0(13)',
 '4.3(8)']

In [121]:
laddr

['임학동 67-7',
 '계산동 942-24번지',
 '계산2동 918-2',
 '작전동 864-39',
 '병방동 422',
 '작전동 863',
 '계산동 1062',
 '계산1동',
 '병방동 426',
 '작전동 925-1']